# Compara Gerenga




In [1]:
import pandas as pd
import psycopg2
import config
import workdays
import datetime
import requests
import yfinance as yf
from yahoofinancials import YahooFinancials
### Acoes




In [2]:
x= ['Acao','Futuro','Titulos Publicos']
y=0
for name in x:
    df=pd.read_excel("G:\\Trading\\K11\\Gerencial\\Previa_Gerencial_nova2.xlsx",sheet_name=f"{name}")

    trade=df[['str_estrategia.1','str_fundo.1','str_codigo.1','str_serie.1','dbl_lote.1','pnl.1']]
    if  name !='Futuro':
        df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','over cdi']]
    else:
        df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','pnl']]
        df=df.rename(columns={'pnl':'over cdi'})

    df=df[df['over cdi']!=0]
    trade=trade.fillna(0)
    trade=trade[trade['pnl.1']!=0]
    trade=trade.rename(columns={'str_estrategia.1':'str_estrategia','str_fundo.1':'str_fundo','str_codigo.1':'str_codigo','str_serie.1':'str_serie','dbl_lote.1':'dbl_lote','pnl.1':'over cdi'})


    full=pd.concat([df, trade]).groupby(['str_estrategia','str_fundo','str_serie']).sum().reset_index()
    
    if name=='Futuro':
        query= f"SELECT dte_data,str_mercado,str_mesa ,str_estrategia,str_codigo,str_fundo,str_serie,dbl_qteinicio,dbl_qtefinal as dbl_lote, dbl_ajcarteira,dbl_ajtotal,dbl_ganho as ganho \
         FROM tbl_gerencialajustes where str_mesa ='Kapitalo 11.1' and dte_data='2022-02-21' "
    else:   
        query= f"SELECT str_fundo, dte_data, str_mercado, str_mesa, str_estrategia, str_codigo, str_serie, dbl_qteinicio, dbl_qtefinal as dbl_lote, dbl_puanterior, dbl_pufinal, dbl_financeiro, dbl_ganho as ganho \
        FROM public.tbl_gerencialvalorizacao where dte_data='2022-02-18' and str_mesa='Kapitalo 11.1' and str_mercado='{name}'"


    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')

    ga=pd.read_sql(query,db_conn_test)
    print(name)
    if name =='Titulos Publicos':
        full['str_serie']=full['str_serie'].astype(str)
    
    compara=pd.merge(ga,full, on =['str_serie','dbl_lote','str_fundo','str_estrategia'], how ='left')

    compara['dif']=compara['ganho']-compara['over cdi']

    compara['error']=(compara['dif']/compara['ganho'])*100
    
    compara.to_excel('compara_gerenga.xlsx',sheet_name=f"{name}")

Acao


KeyError: "None of [Index(['str_estrategia.1', 'str_fundo.1', 'str_codigo.1', 'str_serie.1',\n       'dbl_lote.1', 'pnl.1'],\n      dtype='object')] are in the [columns]"

In [ ]:
name='Futuro'
df=pd.read_excel("G:\\Trading\\K11\\Gerencial\\Previa_Gerencial_nova2.xlsx",sheet_name=f"{name}")

trade=df[['str_estrategia.1','str_fundo.1','str_codigo.1','str_serie.1','dbl_lote.1','pnl.1']]
if  name !='Futuro':
    df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','over cdi']]
else:
    df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','pnl']]
    df=df.rename(columns={'pnl':'over cdi'})


    
df=df.fillna(0)
df=df[df['over cdi']!=0]
# display(df)
trade=trade.fillna(0)
trade=trade[trade['pnl.1']!=0]
trade=trade.rename(columns={'str_estrategia.1':'str_estrategia','str_fundo.1':'str_fundo','str_codigo.1':'str_codigo','str_serie.1':'str_serie','dbl_lote.1':'dbl_lote','pnl.1':'over cdi'})


full=pd.concat([df, trade]).groupby(['str_estrategia','str_fundo','str_serie','str_codigo',]).sum().reset_index()
# display(full)
query= f"SELECT * FROM tbl_gerencialajustes where str_mesa ='Kapitalo 11.1' and dte_data='2022-02-21' "


db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')
db_conn_kapv1 = psycopg2.connect(
    host=config.DB_KAPV1_HOST,
    dbname=config.DB_KAPV1_NAME,
    user=config.DB_KAPV1_USER,
    password=config.DB_KAPV1_PASS,
)
ga=pd.read_sql(query,db_conn_test)

# ga['str_codigo']=ga['str_codigo'].apply(lambda x: x.replace('-',""))
ga.to_excel('gerenga_fut.xlsx')
# compara=pd.merge(ga,full, on =['str_serie','dbl_lote','str_fundo','str_estrategia'], how ='left')

# compara['dif']=compara['ganho']-compara['over cdi']

# compara['error']=(compara['dif']/compara['ganho'])*100
# # compara
# full.to_excel('compara_gerenga.xlsx')

# Comparativo Gerencial



In [4]:
import pandas as pd
import psycopg2
import config
import workdays
import datetime
import requests
import yfinance as yf
from yahoofinancials import YahooFinancials
import xlsxwriter
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import matplotlib.pyplot as plt


# option = webdriver.ChromeOptions()
# option.add_argument('headless')
# driver = webdriver.Chrome('C:\\Users\\joao.ramalho\\Downloads\\chromedriver_win\\chromedriver.exe',options=option)
# driver.get('https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/mercado-de-derivativos/indicadores/indicadores-financeiros/')
# element=WebDriverWait(driver,5).until(EC.visibility_of_element_located((By.XPATH ,'//*[@id="conteudo-principal"]/div[4]/div/div/div[2]/div[3]/table/tbody/tr/td')))


# dolar_spot=float(element.text.replace(",","."))
holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')
dt = datetime.date.today()
dt_1 = workdays.workday(dt, -1, holidays_br)
dt_2 = workdays.workday(dt, -2, holidays_b3)
writer = pd.ExcelWriter("G:\\Trading\\K11\\Gerencial\\Comparativo_Gerencial _test.xlsx", engine='xlsxwriter')
db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')

query=f"select * from tbl_indicadores1 where str_indicador='PtaxCupomLimpo' and dte_data='{dt_2.strftime('%Y-%m-%d')}'"
dolar_spot_1=pd.read_sql(query,db_conn_test).iloc[0]['dbl_preco']






In [7]:
query= f"SELECT  * \
        FROM tbl_gerencialajustes where dte_data='{dt_2.strftime('%Y-%m-%d')}' and str_mesa ='Kapitalo 11.1' and str_mercado='Futuro'"
db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')
ga=pd.read_sql(query,db_conn_test)
ga.to_excel("gerenga_fut.xlsx")

In [ ]:
##Acoes
name='Acao'
df=pd.read_excel("G:\\Trading\\K11\\Gerencial\\Previa_Gerencial_nova2.xlsx",sheet_name=f"{name}")

trade=df[['str_estrategia.1','str_fundo.1','str_serie.1','dbl_lote.1','pnl.1']]
if  name !='Futuro':
    df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','over cdi']]
else:
    df=df[['str_estrategia','str_fundo','str_serie','dbl_lote','pnl']]
    df=df.rename(columns={'pnl':'over cdi'})

df=df[df['over cdi']!=0]
trade=trade.fillna(0)
# trade=trade[trade['pnl.1']!=0]
trade=trade.rename(columns={'str_estrategia.1':'str_estrategia','str_fundo.1':'str_fundo','str_serie.1':'str_serie','dbl_lote.1':'dbl_lote','pnl.1':'over cdi'})


full=pd.concat([df, trade]).groupby(['str_estrategia','str_fundo','str_serie']).sum().reset_index()

if name=='Futuro':
    query= f"SELECT dte_data,str_mercado,str_mesa ,str_estrategia,str_codigo,str_fundo,str_serie,dbl_qteinicio,dbl_qtefinal as dbl_lote, dbl_ajcarteira,dbl_ajtotal,dbl_ganho as ganho \
     FROM tbl_gerencialajustes where str_mesa ='Kapitalo 11.1' and dte_data= '{dt_1.strftime('%Y-%m-%d')}'  "
else:   
    query= f"SELECT str_fundo, dte_data, str_mercado, str_mesa, str_estrategia, str_codigo, str_serie, dbl_qteinicio, dbl_qtefinal as dbl_lote, dbl_puanterior, dbl_pufinal, dbl_financeiro, dbl_ganho as ganho \
    FROM public.tbl_gerencialvalorizacao where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa='Kapitalo 11.1' and str_mercado='{name}' "


db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')

ga=pd.read_sql(query,db_conn_test)

# print(ga)
if name =='Titulos Publicos':
    full['str_serie']=full['str_serie'].astype(str)

compara_1=pd.merge(ga,full, on =['str_serie','str_fundo','str_estrategia','dbl_lote',], how ='left')


compara_1['valor d1']=compara_1['dbl_puanterior']*compara_1['dbl_qteinicio']

compara_1['ganho']=compara_1['ganho']-(compara_1['valor d1']*(((1+ (11.65/100))**(1/252))-1))
compara_1['dif']=abs(abs(compara_1['ganho'])-abs(compara_1['over cdi']))
# compara_1['error']=(compara_1['dif']/compara_1['ganho'])*100

dif_acao=compara_1
print(dif_acao)

compara_1.to_excel(writer,sheet_name=f"{name}")






graph=compara_1[['str_serie','dif']]

graph=graph.groupby('str_serie').agg({'dif':sum}).sort_values(by='dif', ascending=False)
graph.head().plot(kind='bar')

In [ ]:
#Futuros

name='Futuro'
df=pd.read_excel("G:\\Trading\\K11\\Gerencial\\Previa_Gerencial_nova2.xlsx",sheet_name=f"{name}")

trade=df[['str_estrategia.1','str_fundo.1','str_codigo.1','str_serie.1','dbl_lote.1','pnl.1']]
if  name !='Futuro':
    df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','over cdi']]
else:
    df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','over cdi']]

trade=trade.fillna(0)
# trade=trade[trade['pnl.1']!=0]
trade=trade.rename(columns={'str_estrategia.1':'str_estrategia','str_fundo.1':'str_fundo','str_codigo.1':'str_codigo','str_serie.1':'str_serie','dbl_lote.1':'dbl_lote','pnl.1':'over cdi'})


full=pd.concat([df, trade]).groupby(['str_estrategia','str_fundo','str_codigo','str_serie']).sum().reset_index()
display(full)
if name=='Futuro':
    query= f"SELECT dte_data,str_mercado,str_mesa ,str_estrategia,str_codigo,str_fundo,str_serie,dbl_qteinicio,dbl_qtefinal as dbl_lote,dbl_ganho as ganho \
         FROM tbl_gerencialajustes where str_mesa ='Kapitalo 11.1' and dte_data='{dt_1.strftime('%Y-%m-%d')}'"
    
    query2=f"SELECT  dte_data,str_mercado,str_mesa ,str_estrategia,str_codigo,str_fundo,str_serie,dbl_qtefinal as dbl_lote,dbl_ganho as ganho  \
     FROM tbl_gerencialajustesvirtuais where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa ='Kapitalo 11.1' and str_mercado='Futuro'"

else:   
    query= f"SELECT str_fundo, dte_data, str_mercado, str_mesa, str_estrategia, str_codigo, str_serie, dbl_qteinicio, dbl_qtefinal as dbl_lote, dbl_puanterior, dbl_pufinal, dbl_financeiro, dbl_ganho as ganho \
    FROM public.tbl_gerencialvalorizacao where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa='Kapitalo 11.1' and str_mercado='{name}'"


db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')

ga1=pd.read_sql(query,db_conn_test)

ga2=pd.read_sql(query2,db_conn_test)
ga2["str_codigo"] = ga2["str_codigo"].apply(
    lambda x: x.replace("MES@CME", "MES")
)
ga2['ganho']=ga2['ganho']*dolar_spot

ga=pd.concat([ga1,ga2]).reset_index()

ga['str_fundo']=ga['str_fundo'].apply(lambda x: x.replace('KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO B USD',"KAPITALO CLASS B"))
ga['str_fundo']=ga['str_fundo'].apply(lambda x: x.replace('KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO K USD',"KAPITALO CLASS K"))
ga['str_fundo']=ga['str_fundo'].apply(lambda x: x.replace('KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO Q USD',"KAPITALO CLASS Q"))


ga['dbl_lote']=ga.apply(lambda row:-row['dbl_lote'] if (row['str_codigo']=='DI1' or row['str_codigo']=='DDI') else row['dbl_lote'],axis=1)
ga=ga.groupby(['dte_data','str_codigo','str_mercado','str_mesa','str_estrategia','str_fundo','str_serie']).agg({'ganho':sum,'dbl_lote':sum})


if name =='Titulos Publicos':
    full['str_serie']=full['str_serie'].astype(str)

compara_2=pd.merge(ga,full, on =['str_serie','str_codigo','str_fundo','str_estrategia'], how ='left')
# 
compara_2['str_fundo']=compara_2['str_fundo'].apply(lambda x: x.replace('KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO B USD',"KAPITALO CLASS B"))
compara_2['str_fundo']=compara_2['str_fundo'].apply(lambda x: x.replace('KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO K USD',"KAPITALO CLASS K"))
compara_2['str_fundo']=compara_2['str_fundo'].apply(lambda x: x.replace('KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO Q USD',"KAPITALO CLASS Q"))

compara_2["str_codigo"] = compara_2["str_codigo"].apply(
    lambda x: x.replace("MES@CME", "MES")
)
def cdi_futuro(name,fundo,serie):

    if name in ['MES','ES','ZN','TY']:
        if fundo == "KAPITALO CLASS B":
            fundo='KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO B USD'
        elif fundo == "KAPITALO CLASS K":
             fundo='KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO K USD'
        else:
            fundo=fundo
        if name=='MES':
            name="MES@CME"
        query= f"SELECT  * \
        FROM tbl_gerencialajustesvirtuais where dte_data='{dt_2.strftime('%Y-%m-%d')}' and str_mesa ='Kapitalo 11.1' and str_mercado='Futuro' and str_codigo='{name}' and str_fundo='{fundo}' and str_serie='{serie}' "
        db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')
        ga=pd.read_sql(query,db_conn_test)
        ga['str_codigo']=ga['str_codigo'].apply(lambda x: x.replace('@CME',''))
        # ga=ga[ga['str_codigo']==name]
        cdi=((1+0.1165)**(1/252))-1
        try:
            cust=ga.iloc[0]['dbl_ganho']* cdi * dolar_spot_1
            return round(cust,2)
        except:
            return 0


    else:
        query= f"SELECT  * \
        FROM tbl_gerencialajustes where dte_data='{dt_2.strftime('%Y-%m-%d')}' and str_mesa ='Kapitalo 11.1' and str_mercado='Futuro' and str_codigo='{name}' and str_fundo='{fundo}' and str_serie='{serie}' "
        db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')
        ga=pd.read_sql(query,db_conn_test)
        # ga=ga[ga['str_codigo']==name]
        cdi=((1+0.1165)**(1/252))-1
        try:
            cust=ga.iloc[0]['dbl_ganho']* cdi
            return round(cust,2)
        except:
            return 0

compara_2['ganho']=compara_2.apply(lambda row: row['ganho']- abs(cdi_futuro(row['str_codigo'],row['str_fundo'],row['str_serie'])),axis=1)

compara_2['dif']=abs(abs(compara_2['ganho'])-abs(compara_2['over cdi']))


compara_2=compara_2.fillna(0)
dif_futuro=compara_2
print(dif_futuro)


compara_2.to_excel(writer,sheet_name=f"{name}")


graph=compara_2[['str_serie','dif']]

graph=graph.groupby('str_serie').agg({'dif':sum}).sort_values(by='dif', ascending=False)
graph.head().plot(kind='bar')



In [ ]:
#Titulos Publicos

name='Titulos Publicos'
df=pd.read_excel("G:\\Trading\\K11\\Gerencial\\Previa_Gerencial_nova2.xlsx",sheet_name=f"{name}")


try:
    trade=df[['str_estrategia.1','str_fundo.1','str_codigo.1','str_serie.1','dbl_lote.1','dbl_putrade','pnl.1']]
except:
    trade=pd.DataFrame(columns=['str_estrategia.1','str_fundo.1','str_codigo.1','str_serie.1','dbl_lote.1','pnl.1'])
    
if  name !='Futuro':
    df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','over cdi']]
else:
    df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','pnl']]
    df=df.rename(columns={'pnl':'over cdi'})

# df=df[df['over cdi']!=0]
# trade=trade.fillna(0)
# trade=trade[trade['pnl.1']!=0]


trade=trade.rename(columns={'str_estrategia.1':'str_estrategia','str_fundo.1':'str_fundo','str_codigo.1':'str_codigo','str_serie.1':'str_serie','dbl_lote.1':'dbl_lote','pnl.1':'over cdi'})

# trade=trade.groupby(['str_estrategia','str_fundo','str_serie']).agg({'over cdi':sum,'dbl_putrade'})

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# aux=trade
# aux['notional']=aux['dbl_lote']*aux['dbl_putrade']
# aux=aux.groupby(['str_estrategia','str_fundo','str_serie']).agg({'notional':sum,'dbl_lote':sum}).reset_index()
# aux['medio']=aux['notional']/aux['dbl_lote']
# aux=aux[['str_fundo','str_serie','dbl_lote','medio']].set_index(['str_fundo','str_serie'])
# display(aux)




#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

full=pd.concat([df, trade]).groupby(['str_estrategia','str_fundo','str_serie']).sum().reset_index()
display(full)
query= f"SELECT str_fundo, dte_data, str_mercado, str_mesa, str_estrategia, str_codigo, str_serie, dbl_qteinicio, dbl_qtefinal as dbl_lote, dbl_puanterior, dbl_pufinal, dbl_financeiro, dbl_ganho as ganho \
FROM public.tbl_gerencialvalorizacao where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa='Kapitalo 11.1' and str_mercado='{name}'"


db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')

ga=pd.read_sql(query,db_conn_test)
if name =='Titulos Publicos':
    full['str_serie']=full['str_serie'].astype(int)
    full['str_serie']=full['str_serie'].astype(str)

compara_3=pd.merge(ga,full, on =['str_serie','str_fundo','str_estrategia'], how ='left')

compara_3['valor d1']=compara_3['dbl_puanterior']*compara_3['dbl_qteinicio']

compara_3['ganho']=compara_3['ganho']-(compara_3['valor d1']*(((1+ (10.65/100))**(1/252))-1))
compara_3['dif']=abs(abs(compara_3['ganho'])-abs(compara_3['over cdi']))
compara_3['error']=(compara_3['dif']/compara_3['ganho'])*100

# print(compara_3)
compara_3.to_excel(writer,sheet_name=f"{name}")

dif_titpub=compara_3

graph=compara_3[['str_serie','dif']]

graph=graph.groupby('str_serie').agg({'dif':sum}).sort_values(by='dif', ascending=False)
graph.head().plot(kind='bar')


In [ ]:
### Termos 

name='Termos'

df=pd.read_excel("G:\\Trading\\K11\\Gerencial\\Previa_Gerencial_nova2.xlsx",sheet_name=f"{name}")
display(df)
try:
    trade=df[['str_estrategia.1','str_fundo.1','str_codigo.1','str_serie.1','dbl_lote.1','pnl.1']]
except:
    trade=pd.DataFrame(columns=['str_estrategia.1','str_fundo.1','str_codigo.1','str_serie.1','dbl_lote.1','pnl.1'])
    
trade=trade.rename(columns={'str_estrategia.1':'str_estrategia','str_fundo.1':'str_fundo','str_codigo.1':'str_codigo','str_serie.1':'str_serie','dbl_lote.1':'dbl_lote','pnl.1':'over cdi'})


if  name !='Futuro':
    df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','over cdi']]
else:
    df=df[['str_estrategia','str_fundo','str_codigo','str_serie','dbl_lote','pnl']]
    df=df.rename(columns={'pnl':'over cdi'})


full=pd.concat([df, trade]).groupby(['str_estrategia','str_fundo','str_serie']).sum().reset_index()


query= f"SELECT str_fundo, dte_data, str_mercado, str_mesa, str_estrategia, str_codigo, str_serie, dbl_qteinicio, dbl_qtefinal as dbl_lote, dbl_puanterior, dbl_pufinal, dbl_financeiro, dbl_ganho as ganho \
FROM public.tbl_gerencialvalorizacao where dte_data='{dt_1.strftime('%Y-%m-%d')}' and str_mesa='Kapitalo 11.1' and str_mercado='Termo de Acao'"


db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user='k_geral', password='Kapitalo@01')

ga=pd.read_sql(query,db_conn_test)

compara_4=pd.merge(ga,full, on =['str_serie','str_fundo','str_estrategia'], how ='left')

compara_4['valor d1']=compara_4['dbl_puanterior']*compara_4['dbl_qteinicio']

compara_4['ganho']=compara_4['ganho']-(compara_4['valor d1']*(((1+ (11.65/100))**(1/252))-1))
compara_4['dif']=abs(abs(compara_4['ganho'])-abs(compara_4['over cdi']))


display(compara_4)
compara_4.to_excel(writer,sheet_name=f"{name}")

dif_termos=compara_4
print(dif_termos)

graph=compara_4[['str_serie','dif']]

graph=graph.groupby('str_serie').agg({'dif':sum}).sort_values(by='dif', ascending=False)
graph.head().plot(kind='bar')
writer.save()

In [ ]:
## Relatório 

diferenca=pd.concat([dif_acao,
dif_futuro,
dif_titpub,
dif_termos,
])[['str_estrategia','dif']].groupby('str_estrategia').agg({'dif':sum})

##Diferença por estratégia

diferenca.plot(kind='bar')

print("Diferença total: R$: "+ str(diferenca['dif'].sum()))

